#### Initialize ####

In [1]:
function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [2]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [ ]:
(pwd).path

In [14]:
terraform init #-upgrade

Initializing the backend...
Initializing modules...
Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v5.70.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


#### Workflow ####

In [3]:
terraform fmt

main.tf


In [4]:
terraform validate

Success! The configuration is valid.



In [7]:
###      ###
### PLAN ###
###      ###

tf plan -out=tfplan
terraform show -json tfplan > plan.json

data.aws_availability_zones.available: Reading...
data.aws_availability_zones.available: Read complete after 1s [id=us-east-2]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # module.vpc.aws_default_network_acl.this[0] will be created
  + resource "aws_default_network_acl" "this" {
      + arn                    = (known after apply)
      + default_network_acl_id = (known after apply)
      + id                     = (known after apply)
      + owner_id               = (known after apply)
      + tags                   = {
          + "Example" = "example_tag"
          + "Name"    = "TF-VPC1-default"
        }
      + tags_all               = {
          + "Example" = "example_tag"
          + "Name"    = "TF-VPC1-default"
        }
      + vpc_id                 = (known after apply)

      + egress {
          + action          = 

In [8]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

Type: aws_default_network_acl
TF name: this
Cloud name: 
Name tag: TF-VPC1-default

Type: aws_default_route_table
TF name: default
Cloud name: 
Name tag: TF-VPC1-default

Type: aws_default_security_group
TF name: this
Cloud name: 
Name tag: TF-VPC1-default

Type: aws_internet_gateway
TF name: this
Cloud name: 
Name tag: TF-VPC1

Type: aws_route
TF name: public_internet_gateway
Cloud name: 
Name tag: 

Type: aws_route_table
TF name: private
Cloud name: 
Name tag: TF-VPC1-private-us-east-2a

Type: aws_route_table
TF name: private
Cloud name: 
Name tag: TF-VPC1-private-us-east-2b

Type: aws_route_table
TF name: private
Cloud name: 
Name tag: TF-VPC1-private-us-east-2c

Type: aws_route_table
TF name: public
Cloud name: 
Name tag: TF-VPC1-public

Type: aws_route_table_association
TF name: private
Cloud name: 
Name tag: 

Type: aws_route_table_association
TF name: private
Cloud name: 
Name tag: 

Type: aws_route_table_association
TF name: private
Cloud name: 
Name tag: 

Type: aws_route_tabl

In [9]:
###       ###
### APPLY ###
###       ###

try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

data.aws_availability_zones.available: Reading...
data.aws_availability_zones.available: Read complete after 1s [id=us-east-2]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # module.vpc.aws_default_network_acl.this[0] will be created
  + resource "aws_default_network_acl" "this" {
      + arn                    = (known after apply)
      + default_network_acl_id = (known after apply)
      + id                     = (known after apply)
      + owner_id               = (known after apply)
      + tags                   = {
          + "Example" = "example_tag"
          + "Name"    = "TF-VPC1-default"
        }
      + tags_all               = {
          + "Example" = "example_tag"
          + "Name"    = "TF-VPC1-default"
        }
      + vpc_id                 = (known after apply)

      + egress {
          + action          = 

In [ ]:
###         ###
### DESTROY ###
###         ###

try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

#### Info ####

In [10]:
terraform version

Terraform v1.9.5
on windows_amd64
+ provider registry.terraform.io/hashicorp/aws v5.70.0

Your version of Terraform is out of date! The latest version
is 1.9.7. You can update by downloading from https://www.terraform.io/downloads.html


In [11]:
#Root module configuration
cat .\main.tf

terraform {
  required_version = ">= 1.9.5, < 2.0"

  required_providers {
    aws = {
      source = "hashicorp/aws"
    }
  }
}

provider "aws" {
  region = var.aws_region
}

# --- --- --- --- --- --- --- --- --- --- #

data "aws_availability_zones" "available" {}

# --- --- --- --- --- --- --- --- --- --- #

locals {
  azs      = slice(data.aws_availability_zones.available.names, 0, 3)
  vpc_cidr = "10.1.0.0/16"
}

# --- --- --- --- --- --- --- --- --- --- #

module "vpc" {
  source  = "terraform-aws-modules/vpc/aws"
  version = "5.13.0"

  name = "TF-${basename(path.cwd)}"
  cidr = local.vpc_cidr

  azs = local.azs

  private_subnets = [for k, v in local.azs : cidrsubnet(local.vpc_cidr, 8, k)]

  public_subnets = [for k, v in local.azs : cidrsubnet(local.vpc_cidr, 8, k + 100)]
  #map_public_ip_on_launch = true

  tags = {
    Example = "example_tag"
  }

}

# --- --- --- --- --- --- --- --- --- --- #

# resource "aws_vpc_endpoint" "s3" {
#   vpc_id       = module.vpc.vpc_id
#   ser

In [12]:
($resources = terraform state list) | Out-Host

data.aws_availability_zones.available
module.vpc.aws_default_network_acl.this[0]
module.vpc.aws_default_route_table.default[0]
module.vpc.aws_default_security_group.this[0]
module.vpc.aws_internet_gateway.this[0]
module.vpc.aws_route.public_internet_gateway[0]
module.vpc.aws_route_table.private[0]
module.vpc.aws_route_table.private[1]
module.vpc.aws_route_table.private[2]
module.vpc.aws_route_table.public[0]
module.vpc.aws_route_table_association.private[0]
module.vpc.aws_route_table_association.private[1]
module.vpc.aws_route_table_association.private[2]
module.vpc.aws_route_table_association.public[0]
module.vpc.aws_route_table_association.public[1]
module.vpc.aws_route_table_association.public[2]
module.vpc.aws_subnet.private[0]
module.vpc.aws_subnet.private[1]
module.vpc.aws_subnet.private[2]
module.vpc.aws_subnet.public[0]
module.vpc.aws_subnet.public[1]
module.vpc.aws_subnet.public[2]
module.vpc.aws_vpc.this[0]


In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [13]:
terraform output

public_subnets = [
  "subnet-0c13b6812ab6e3f89",
  "subnet-0e6c5590894f5f7cd",
  "subnet-07373a76cf5740bbb",
]
vpc_id = "vpc-0a4860611e96d1d85"


#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve